In [1]:
#!/usr/env/python3 

In [1]:
# Importing the necessary modules 
import os 
import cv2 
import imutils 
import pickle 
import numpy as np
import tensorflow as tf 
from imutils import paths
from imutils.paths import list_images
import matplotlib.pyplot as plt 
from bs4 import BeautifulSoup
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam, SGD 
from tensorflow.keras.preprocessing.image import img_to_array 
from tensorflow.keras.preprocessing.image import load_img 
from sklearn.model_selection import  train_test_split 
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, MultiLabelBinarizer, OneHotEncoder  
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.layers import Flatten, Dropout  
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, Activation
from tensorflow.keras.layers import Dense, BatchNormalization  
from tensorflow.keras.layers import MaxPooling2D, SeparableConv2D

In [7]:
# !pip install -U scikit-learn 

In [2]:
# Getting the path to the image directory 
imageDataSet = "dataset/"

# Initialize the initial learning rate, number of epochs to train 
# for and batch size 
INIT_LR = 1e-3; 
EPOCHS = 2
BS = 8 

# Setting the data, and labels 
data = []; 
labels = []; 

# Grab the list of images in our dataset directory, then 
# initialize the list of data, and class images 
imagePaths = list(list_images(imageDataSet)) 

# Loop over the image paths 
for imagePath in imagePaths: 
    # Extract the class label from the filename 
    label = imagePath.split("/")[-2] 
    
    # Load the image, swap color channels, and resize it to be a 
    # fixed 224 X 224 pixel whilte ignoreing aspect ratio 
    image = cv2.imread(imagePath) 
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image = cv2.resize(image, (224, 224)) 
    
    # Update the data and labels lists, respectively 
    data.append(image) 
    labels.append(label) 
    
# Convert the data and labels to numpy arrays while scaling 
# the pixel intensities to the range [0 -1] 
data = np.array(data) / 255.0 
labels = np.array(labels) 

In [3]:
# Perform one hot encoding on the labels 
lb = MultiLabelBinarizer()
labels = lb.fit_transform(labels) 
encodedLabels = to_categorical(labels) 

In [4]:
# Displaying the shape of the input and output variables 
print(f'Input Shape: {data.shape}') 
print(f'Output Shape: {labels.shape}') 

Input Shape: (1500, 224, 224, 3)
Output Shape: (1500, 10)


In [5]:
len(lb.classes_) 

10

In [6]:
# Partition the data into training and testing splits uing 80% 
# of the data for training and the remaining 20% for testing 
X_train, X_test, y_train, y_test = train_test_split(data, encodedLabels, test_size=0.20, random_state=42)

In [7]:
# Initialize the number of epochs to train for, initial learning rate 
# batch size, and image dimensions 
# epochs = 100
epochs = 2
ini_lr = 1e-3 
bs = 32
dim = (224, 224, 3) 
# cov = (3, 3) 
cov = (5, 5)

# Building the neural network model 
class SmallerVGGNet:
    @staticmethod 
    # Creating a function for building the model 
    def build_model(width, height, depth, classes, finalAct="softmax"):
        # Initilize the model along with the input shape to be the 
        # channels last, and the channels dimensions itself 
        model = Sequential() 
        input_shape = (height, width, depth)
        # input_shape = (height, width)
        chan_dim = -1 
      
        # If we are using "channels first", update the input shape 
        # And channels dimension 
        if K.image_data_format() == "channels_first":
            input_shape = (height, width, depth)
            # input_shape = (height, width)
            chan_dim = 1
            
        # Convolution to Rectified to Maxpool2D 
        model.add(SeparableConv2D(32, cov, padding="same", input_shape=input_shape)) 
        model.add(Activation("relu")) 
        model.add(BatchNormalization(axis=chan_dim)) 
        model.add(MaxPooling2D(pool_size=(2, 2))) 
        model.add(Dropout(0.25)) 
        
        # Convolution to Rectified to MaxPool2D 
        model.add(Conv2D(64, cov, padding="same")) 
        model.add(Activation("relu")) 
        model.add(BatchNormalization(axis=chan_dim)) 
        model.add(Conv2D(64, cov, padding="same")) 
        model.add(Activation("relu")) 
        model.add(BatchNormalization(axis=chan_dim)) 
        model.add(MaxPooling2D(pool_size=(2, 2))) 
        model.add(Dropout(0.25)) 
        
        # Convolution to Rectified then MaxPool2D 
        model.add(Conv2D(128, cov, padding="same")) 
        model.add(Activation("relu")) 
        model.add(BatchNormalization(axis=chan_dim)) 
        model.add(Conv2D(128, cov, padding="same")) 
        model.add(Activation("relu")) 
        model.add(BatchNormalization(axis=chan_dim)) 
        model.add(SeparableConv2D(128, cov, padding="same")) 
        model.add(Activation("relu")) 
        model.add(BatchNormalization(axis=chan_dim)) 
        model.add(MaxPooling2D(pool_size=(2, 2))) 
        model.add(Dropout(0.25)) 
        
        # First, and only the Fully connected layers then to Rectified linear unit layers 
        model.add(Flatten()) 
        model.add(Dense(256)) 
        model.add(Activation("relu")) 
        model.add(BatchNormalization()) 
        model.add(Dropout(0.25)) 
        
        # Softmax classified 
        model.add(Dense(classes)) 
        model.add(Activation(finalAct)) 
        
        # Return the constructed neural network 
        return model 
    
    
# Construct the image generator for data augmentation 
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1, 
                             height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, 
                             horizontal_flip=True, fill_mode="nearest") 

# Initialize the model using a sigmoid activation as the final layers in the 
# Network so we can perfom the classification method 
model = SmallerVGGNet.build_model(width=dim[0], height=dim[1], depth=dim[2], 
                                  classes=len(lb.classes_), finalAct="sigmoid") 

# Initialze the optimizer 
opt = Adam(lr=ini_lr, decay=ini_lr / epochs) 

# Compiling the model 
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"]) 

# Model summary 
model.summary() 

2023-03-16 23:40:05.869765: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-16 23:40:06.154082: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 separable_conv2d (Separable  (None, 224, 224, 32)     203       
 Conv2D)                                                         
                                                                 
 activation (Activation)     (None, 224, 224, 32)      0         
                                                                 
 batch_normalization (BatchN  (None, 224, 224, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 112, 112, 32)      0         
                                                        

/home/mbonu/anaconda3/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# Training the model, and generate a weighted file 
H = model.fit(aug.flow(X_train, y_train, batch_size=bs), validation_data=(X_test, y_test), 
                        steps_per_epoch=len(X_train) // bs, 
                        epochs=epochs, verbose=1) 

Epoch 1/2


In [15]:
# Initialize the training daa augmentaion object 
trainAug = ImageDataGenerator(
    rotation_range=15, 
    fill_mode="nearest")

# Load the VGG16 network, ensuring the head FC layer sets are left 
# off 
baseModel = VGG16(weights="imagenet", include_top=False, 
                  input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the 
# base model 
headModel = baseModel.output 
headModel = AveragePooling2D(pool_size=(4, 4))(headModel) 
headModel = Flatten(name="flatten")(headModel) 
headModel = Dense(64, activation="relu")(headModel) 
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel) 

# Place the head FC model on top of the base model 
model = Model(inputs=baseModel.input, outputs=headModel) 

In [16]:
# Compile our model 
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS) 
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=['accuracy']) 

/home/mbonu/anaconda3/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# Training the model 
H = model.fit_generator(
    trainAug.flow(trainX, trainY, batch_size=BS), 
    steps_per_epoch=len(trainX) // BS, 
    validation_data=(testX, testY), 
    validation_steps=len(testX) // BS, 
    epoch=EPOCHS) 